In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import *
from keras.models import Sequential,Model
import re

Using TensorFlow backend.


In [42]:
train = pd.read_csv("Train.csv")
test = pd.read_csv("Test.csv")
train.head()

,question,answer_text,distractor
0,Meals can be served,in rooms at 9:00 p. m.,"'outside the room at 3:00 p. m.', 'in the dini..."
1,It can be inferred from the passage that,The local government can deal with the problem...,"'If some tragedies occur again ', ' relevant d..."
2,The author called Tommy 's parents in order to,help them realize their influence on Tommy,"'blame Tommy for his failing grades', 'blame T..."
3,It can be inferred from the passage that,the writer is not very willing to use idioms,'idioms are the most important part in a langu...
4,How can we deal with snake wounds according to...,Stay calm and do n't move .,'Cut the wound and suck the poison out .'


In [43]:
test = test.values

In [3]:
train = train.values
answers = {}
distractors = {}
count = 0
for x in range(train.shape[0]):
    answers[train[x][0]] = train[x][1]
    a=[]
    for y in train[x][2].split(", "):
        a.append(str(y[1:-1]))
    distractors[train[x][0]] = a
    count = count+1

In [4]:
distractors["Meals can be served"]

['outside the room at 3:00 p. m.',
 'in the dining - room at 6:00 p. m.',
 'in the dining - room from 7:30 a. m. to 9:15 p. m.']

In [5]:
def clean_text(sentence):
    sentence = sentence.lower()
    sentence = re.sub("[^a-z0-9]+"," " , sentence)
    sentence = sentence.split()

    sentence = [s for s in sentence if((len(s)>1) or (re.match("[0-9]+",s) is not None))]
    sentence = " ".join(sentence)

    return sentence

In [6]:
# Clean all the captions
a={}
d={}
for key , dist_list in distractors.items():
    for i in range(len(dist_list)):
        dist_list[i] = clean_text(dist_list[i])
    answer=clean_text(answers[key])
    key=clean_text(key)
    a[key]=answer
    d[key]=dist_list
answers=a
distractors=d

In [7]:
distractors["meals can be served"]

['outside the room at 3 00',
 'in the dining room at 6 00',
 'in the dining room from 7 30 to 9 15']

In [8]:
with open("answers.txt","w") as f:
    f.write(str(answers))
with open("distractors.txt","w") as f:
    f.write(str(distractors))

In [9]:
vocab = set()
for key in answers.keys():
    [vocab.update(key.split())]
    [vocab.update(answers[key].split())]
    [vocab.update(sentence.split()) for sentence in distractors[key]]

    
print(len(vocab))

21459


In [10]:
total = []
for key in answers.keys():
    [total.append(i) for i in key.split()]
    [total.append(i) for i in answers[key].split()]
    [total.append(i) for des in distractors[key] for i in des.split()]

print(len(total))

718584


In [11]:
import collections
counter = collections.Counter(total)
freq_cnt = dict(counter)

sorted_freq_cnt = sorted(freq_cnt.items(),reverse = True,key = lambda x:x[1])

threshold =10
sorted_freq_cnt = [x for x in sorted_freq_cnt if x[1]>threshold]
total_words = [x[0] for x in sorted_freq_cnt]
print(len(sorted_freq_cnt))

4723


In [12]:
train_distractors = {}
for key in distractors.keys():
    train_distractors[key] = []
    for dist in distractors[key]:
        dist_to_append = "StartSeq " + dist + " EndSeq"
        train_distractors[key].append(dist_to_append)
        
print(train_distractors["meals can be served"])

['StartSeq outside the room at 3 00 EndSeq', 'StartSeq in the dining room at 6 00 EndSeq', 'StartSeq in the dining room from 7 30 to 9 15 EndSeq']


In [13]:
len(total_words)

4723

In [14]:
word_to_idx = {}
idx_to_word = {}

for i,word in enumerate(total_words):
    word_to_idx[word] = i+1
    idx_to_word[i+1] = word
    
len(word_to_idx)

4723

In [15]:
word_to_idx["StartSeq"]=4724
idx_to_word[4724] = "StartSeq"

word_to_idx["EndSeq"]=4725
idx_to_word[4725] = "EndSeq"

vocab_size = len(word_to_idx)

vocab_size= vocab_size+1

In [16]:
vocab_size

4726

In [17]:
max_len=0
for key in train_distractors.keys():
    for dist in train_distractors[key]:
        max_len = max(max_len,len(dist.split()))
print(max_len)

30


In [18]:
max_q=0
for key in train_distractors.keys():
    max_q = max(max_q,len(key.split()))
print(max_q)

48


In [19]:
max_a = 0
for key in answers.keys():
    max_a = max(max_a,len(answers[key].split()))
print(max_a)

101


In [21]:
def data_generator(train_distractors,answers,word_to_idx,max_len,batch_size):
    X1,X2,X3,y = [],[],[],[]

    n=0
    while True:
        for key,dist_list in train_distractors.items():
            n+=1

            question = key
            answer = answers[key]


            seqq = [word_to_idx[wordQ] for wordQ in question.split() if wordQ in word_to_idx]
            question= pad_sequences([seqq],maxlen=max_q,value=0,padding='post')[0]


            seqa = [word_to_idx[wordA] for wordA in answer.split() if wordA in word_to_idx]
            answer = pad_sequences([seqa],maxlen=max_a,value=0,padding='post')[0]

            for dist in dist_list:
                seq = [word_to_idx[word] for word in dist.split() if word in word_to_idx]
                for i in range(1,len(seq)):
                    xi = seq[0:i]
                    yi = seq[i]

                    xi = pad_sequences([xi],maxlen=max_len,value = 0,padding='post')[0] 
                    yi = to_categorical([yi],num_classes = vocab_size)[0]



                    X1.append(question)
                    X2.append(answer)
                    X3.append(xi)
                    y.append(yi)
                if n==batch_size:
                    yield[[np.array(X1),np.array(X2),np.array(X3)],np.array(y)]
                    X1,X2,X3,y = [],[],[],[]
                    n=0

In [22]:
f=open("glove.6B.100d.txt",encoding="utf8")

In [23]:
embedding_index = {}
for line in f:
  values=line.split()
  word = values[0]
  embedding_index[word]=np.array(values[1:],dtype='float')

In [24]:
embedding_index['apple']

array([-0.5985   , -0.46321  ,  0.13001  , -0.019576 ,  0.4603   ,
       -0.3018   ,  0.8977   , -0.65634  ,  0.66858  , -0.49164  ,
        0.037557 , -0.050889 ,  0.6451   , -0.53882  , -0.3765   ,
       -0.04312  ,  0.51384  ,  0.17783  ,  0.28596  ,  0.92063  ,
       -0.49349  , -0.48583  ,  0.61321  ,  0.78211  ,  0.19254  ,
        0.91228  , -0.055596 , -0.12512  , -0.65688  ,  0.068557 ,
        0.55629  ,  1.611    , -0.0073642, -0.48879  ,  0.45493  ,
        0.96105  , -0.063369 ,  0.17432  ,  0.9814   , -1.3125   ,
       -0.15801  , -0.54301  , -0.13888  , -0.26146  , -0.3691   ,
        0.26844  , -0.24375  , -0.19484  ,  0.62583  , -0.7377   ,
        0.38351  , -0.75004  , -0.39053  ,  0.091498 , -0.36591  ,
       -1.4715   , -0.45228  ,  0.2256   ,  1.1412   , -0.38526  ,
       -0.06716  ,  0.57288  , -0.39191  ,  0.31302  , -0.29235  ,
       -0.96157  ,  0.15154  , -0.21659  ,  0.25103  ,  0.096967 ,
        0.2843   ,  1.4296   , -0.50565  , -0.51374  , -0.4721

In [26]:
def getEmbeddingMatrix():
    emb_dim=100
    matrix = np.zeros((vocab_size,emb_dim))
    for word,idx in word_to_idx.items():
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            matrix[idx] = embedding_vector
    return matrix

In [27]:
embedding_matrix = getEmbeddingMatrix()
embedding_matrix.shape

(4726, 100)

In [28]:
embedding_matrix[4724]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [29]:
print(max_len)

30


In [30]:
input_dist = Input(shape = (max_len,))
input_dist1=  Embedding(input_dim=vocab_size,output_dim=100,mask_zero=True)(input_dist)
input_dist2 = Dropout(0.3)(input_dist1)
input_dist3 = LSTM(256)(input_dist2)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [31]:
input_ques = Input(shape = (max_q,))
input_ques1=  Embedding(input_dim=vocab_size,output_dim=100,mask_zero=True)(input_ques)
input_ques2 = Dropout(0.3)(input_ques1)
input_ques3 = LSTM(256)(input_ques2)

In [32]:
input_ans = Input(shape = (max_a,))
input_ans1=  Embedding(input_dim=vocab_size,output_dim=100,mask_zero=True)(input_ans)
input_ans2 = Dropout(0.3)(input_ans1)
input_ans3 = LSTM(256)(input_ans2)

In [33]:
decoder1 = add([input_dist3,input_ques3,input_ans3])
decoder2 = Dense(512 ,activation = 'relu')(decoder1)
outputs = Dense(vocab_size,activation= 'softmax')(decoder2)

model = Model(inputs = [input_ques,input_ans,input_dist],outputs = outputs)

In [34]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 30)           0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 48)           0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 101)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 30, 100)      472600      input_1[0][0]                    
____________________________________________________________________________________________

In [35]:
model.layers[3].set_weights([embedding_matrix])
model.layers[3].trainable = False  

model.layers[4].set_weights([embedding_matrix])
model.layers[4].trainable = False  
model.layers[5].set_weights([embedding_matrix])
model.layers[5].trainable = False

In [36]:
model.compile(loss='categorical_crossentropy',optimizer = 'adam')

In [37]:
# !mkdir model_weights

In [39]:
from keras.models import load_model
model.load_weights("./model_weights/model_19.h5")

In [40]:
model

In [ ]:
epochs=20
number_of_ques = 64
steps = len(train_distractors)//number_of_ques

for i in range(epochs):
    generator = data_generator(train_distractors,answers,word_to_idx,max_len,number_of_ques)
    model.fit_generator(generator,epochs=1,steps_per_epoch = steps,verbose = 1)
    model.save("./model_weights/model_"+str(i)+".h5")

In [44]:
answers_t = {}
count = 0
for x in range(test.shape[0]):
    answers_t[test[x][0]] = test[x][1]
    count = count+1
    
a={}
for key , answer in answers_t.items():
    answer=clean_text(answers_t[key])
    key=clean_text(key)
    a[key]=answer
answers_t=a

In [45]:
print(answers_t["what the main idea of the text"])

lack of water affects california crops


In [46]:
print(len(answers_t.keys()))

10353


In [48]:
def predict_distractors(X1,X2):
    dists = []
    for j in range(3):
        in_text = "StartSeq"
        for i in range(max_len):
            sequence = [word_to_idx[w] for w in in_text.split() if w in word_to_idx]
            sequence = pad_sequences([sequence],maxlen=max_len,padding = 'post')[0]
            XQ = []
            XA = []
            XI = []
            XQ.append(X1)
            XA.append(X2)
            XI.append(sequence)
            y_pred = model.predict([np.array(XQ),np.array(XA),np.array(XI)])

            if(i<=1):
                y_pred=np.array(y_pred)
                y_pred = y_pred.argsort()
                y_pred=y_pred[0][:]
                y_pred=y_pred[len(y_pred)-1-j]
            else:
                y_pred=y_pred.argmax()
            word = idx_to_word[y_pred]
            in_text += (' ' + word)

            if word == 'EndSeq':
                break
        final_dists = in_text.split()[1:-1]
        final_dists = ' '.join(final_dists)
        dists.append(final_dists)
    return dists

In [49]:
question= clean_text('Why Ranju killed his father?')
answer = clean_text('Ranju was a bad student')




print(question)
print(answer)
seqq = [word_to_idx[wordQ] for wordQ in question.split() if wordQ in word_to_idx]
question= pad_sequences([seqq],maxlen=max_q,value=0,padding='post')[0]

seqa = [word_to_idx[wordA] for wordA in answer.split() if wordA in word_to_idx]
answer = pad_sequences([seqa],maxlen=max_a,value=0,padding='post')[0]

#   question = question.reshape((1,question.shape[0]))
#   answer = answer.reshape((1,answer.shape[0]))


distractor = predict_distractors(question,answer)
distractor = str(distractor)


why ranju killed his father
ranju was bad student



In [50]:

distractor

"['was taken to school', 'did like to go with his mother', 'his father did want to go with his son']"